In [1]:

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import echonet
import tqdm

In [2]:

ds = echonet.datasets.Echo(root = "C:\\Datasets\\ReleasedData", split = "all")
mean, std = echonet.utils.get_mean_and_std(ds)
print(mean, std)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:12<00:00,  1.26it/s]

[32.36437  32.46042  33.180195] [49.859623 49.81081  50.567196]


In [3]:
# From https://github.com/echonet/dynamic/blob/master/scripts/InitializationNotebook.ipynb
data = "C:\\Datasets\\All A4c Videos from Database Query\\DownsampledAVIs"
regression_model_checkpoint = "C:\\Users\\Remote\\Documents\\John\\dynamic\\Weights-20201103T193519Z-001\\Weights\\r2plus1d_18_32_2_pretrained.pt"

# Initialize and Run EF model

frames = 32
period = 2
batch_size = 20
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)


if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(regression_model_checkpoint)
    model.load_state_dict(checkpoint['state_dict'])
    

cuda is available, original weights


In [5]:
unique_firstrun =  "C:\\Datasets\\All A4c Videos from Database Query\\alreadyProcessedFiles-new2.csv"

kwargs = {"target_type": "Filename",
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period
          }

test_ds = echonet.datasets.Echo(split = "external_test", external_test_location = data, **kwargs, clips="all", duplicates_file = unique_firstrun) # unique_fnames) #
#print(test_ds.split, test_ds.fnames)

#ds = echonet.datasets.Echo(root = "C:\\Datasets\\ReleasedData", **kwargs, split = "all", clips = "all")

test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size = 1, num_workers = 5, shuffle = False, pin_memory=(device.type == "cuda"))
# loss, yhat, y = echonet.utils.video.run_epoch(model, test_dataloader, train = False, optim = None, device = device, save_all=True, block_size = 20)

# with tqdm.tqdm(total = len(test_dataloader)) as pbar:
#     for X, outcome in test_dataloader:
#         print(X.shape, outcome)

# with open(output, "w") as g:
#     for (filename, pred) in zip(ds.fnames, yhat):
#         for (i,p) in enumerate(pred):
#             g.write("{},{},{:.4f}\n".format(filename, i, p))

removed duplicates, 81757


In [6]:
torch.cuda.empty_cache()

save_all = True
block_size = 30
train = False
n = 0      # number of videos processed

errorLog = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output_ERROR_FILES.csv")
regression_output = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output.csv")

with open(errorLog, 'w') as errors:
    with open(regression_output, 'w') as g:
        with torch.set_grad_enabled(train):
            with tqdm.tqdm(total=len(test_dataloader)) as pbar:
                for i, (X, outcome) in enumerate(test_dataloader):
                    try:
                        #print(X.shape, flush = True)

                        #y.append(outcome.numpy())
                        #X = X.to(device)
                        #outcome = outcome.to(device)

                        average = (len(X.shape) == 6)
                        if average:
                            batch, n_clips, c, f, h, w = X.shape
                            X = X.view(-1, c, f, h, w)

                        if block_size is None:
                            outputs = model(X)
                        else:

                            for j in range(0, X.shape[0], block_size):
                                thisX = X[j:(j + block_size)].to(device)
                                thisOutput = model(thisX).detach().cpu()

                                for k, l in enumerate(thisOutput):
                                    #print(outcome[0], (j + k), l)
                                    g.write(outcome[0] + "," + str(j + k) + "," + str(l.item()) + "\n")

                        n += X.size(0)

                        pbar.set_postfix_str("{:.2f} ({:.2f})".format( n, i))
                        pbar.update()

                        #print(s1, s2, total)
                    except:
                        print(X.shape, flush = True)
                        print("something did not go right with", outcome)
                        errors.write(outcome[0] + "\n")

                    torch.cuda.empty_cache()
                    #    print("cache cleared!", torch.cuda.memory_allocated(0))



100%|██████████████████████████████████████████████████| 26693/26693 [12:39:28<00:00,  1.71s/it, 2902621.00 (26692.00)]


In [ ]:
# Load already processed, to ignore in dataloader

In [7]:
#Make a file with name of all files already run

firstrun = "C:\\Datasets\\All A4c Videos from Database Query\\allA4c_ef_output_thirdrun.csv"
unique_firstrun =  "C:\\Datasets\\All A4c Videos from Database Query\\alreadyProcessedFiles-new2.csv"
unique_firstrun_previous =  "C:\\Datasets\\All A4c Videos from Database Query\\alreadyProcessedFiles-new.csv"

unique_fnames = []

with open(unique_firstrun_previous) as previous:
    with open(firstrun, 'r') as f:
        with open(unique_firstrun, 'w') as fout:
            for line in previous.readlines():
                fout.write(line)
                unique_fnames.append(line.strip())
            
            print("previous files loaded")#, unique_fnames)
            
            for line in f.readlines():
                fname = line.split(',')[0]
                if fname not in unique_fnames:
                    unique_fnames.append(fname)
                    fout.write(fname + "\n")

print(len(unique_fnames))
    

previous files loaded
81757


In [ ]:
# Previous runs

In [7]:
save_all = True
block_size = 20
train = False
n = 0      # number of videos processed

errorLog = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output_ERROR_FILES.csv")
regression_output = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output.csv")

with open(errorLog, 'w') as errors:
    with open(regression_output, 'w') as g:
        with torch.set_grad_enabled(train):
            with tqdm.tqdm(total=len(test_dataloader)) as pbar:
                for i, (X, outcome) in enumerate(test_dataloader):
                    try:
                        #print(X.shape, flush = True)

                        #y.append(outcome.numpy())
                        #X = X.to(device)
                        #outcome = outcome.to(device)

                        average = (len(X.shape) == 6)
                        if average:
                            batch, n_clips, c, f, h, w = X.shape
                            X = X.view(-1, c, f, h, w)

                        if block_size is None:
                            outputs = model(X)
                        else:

                            for j in range(0, X.shape[0], block_size):
                                thisX = X[j:(j + block_size)].to(device)
                                thisOutput = model(thisX).detach().cpu()

                                for k, l in enumerate(thisOutput):
                                    #print(outcome[0], (j + k), l)
                                    g.write(outcome[0] + "," + str(j + k) + "," + str(l.item()) + "\n")

                        n += X.size(0)

                        pbar.set_postfix_str("{:.2f} ({:.2f})".format( n, i))
                        pbar.update()

                        #print(s1, s2, total)
                    except:
                        print(X.shape, flush = True)
                        print("something did not go right with", outcome)
                        errors.write(outcome[0] + "\n")

                    torch.cuda.empty_cache()
                    #    print("cache cleared!", torch.cuda.memory_allocated(0))



  1%|▎                                                      | 649/108450 [19:12<49:13:51,  1.64s/it, 70238.00 (648.00)]

torch.Size([123, 3, 32, 112, 112])
something did not go right with ['1BTBOVQH.avi']


 24%|██████████▉                                   | 25664/108450 [12:07:21<25:23:35,  1.10s/it, 2757264.00 (25664.00)]

torch.Size([147, 3, 32, 112, 112])
something did not go right with ['3VPX4K99.avi']


 24%|██████████▉                                   | 25664/108450 [12:07:24<39:06:26,  1.70s/it, 2757264.00 (25664.00)]


RuntimeError: CUDA error: out of memory

In [6]:
torch.cuda.empty_cache()

save_all = True
block_size = 30
train = False
n = 0      # number of videos processed

errorLog = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output_ERROR_FILES.csv")
regression_output = os.path.join("C:\\Datasets\\All A4c Videos from Database Query", "allA4c_ef_output.csv")

with open(errorLog, 'w') as errors:
    with open(regression_output, 'w') as g:
        with torch.set_grad_enabled(train):
            with tqdm.tqdm(total=len(test_dataloader)) as pbar:
                for i, (X, outcome) in enumerate(test_dataloader):
                    try:
                        #print(X.shape, flush = True)

                        #y.append(outcome.numpy())
                        #X = X.to(device)
                        #outcome = outcome.to(device)

                        average = (len(X.shape) == 6)
                        if average:
                            batch, n_clips, c, f, h, w = X.shape
                            X = X.view(-1, c, f, h, w)

                        if block_size is None:
                            outputs = model(X)
                        else:

                            for j in range(0, X.shape[0], block_size):
                                thisX = X[j:(j + block_size)].to(device)
                                thisOutput = model(thisX).detach().cpu()

                                for k, l in enumerate(thisOutput):
                                    #print(outcome[0], (j + k), l)
                                    g.write(outcome[0] + "," + str(j + k) + "," + str(l.item()) + "\n")

                        n += X.size(0)

                        pbar.set_postfix_str("{:.2f} ({:.2f})".format( n, i))
                        pbar.update()

                        #print(s1, s2, total)
                    except:
                        print(X.shape, flush = True)
                        print("something did not go right with", outcome)
                        errors.write(outcome[0] + "\n")

                    torch.cuda.empty_cache()
                    #    print("cache cleared!", torch.cuda.memory_allocated(0))



 41%|███████████████████▍                            | 18243/44937 [8:52:01<18:13:58,  2.46s/it, 2021596.00 (18242.00)]

torch.Size([240, 3, 32, 112, 112])
something did not go right with ['BSILDU76.avi']


 41%|███████████████████▍                            | 18243/44937 [8:52:05<12:58:34,  1.75s/it, 2021596.00 (18242.00)]


RuntimeError: CUDA error: out of memory